In [1]:
# Schritt 1: Abhängigkeiten installieren
%pip install --quiet psycopg2-binary gradio pillow opencv-python-headless python-dotenv numpy



[notice] A new release of pip available: 22.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Schritt 2: Imports & Settings
import io
import uuid
import json
import hashlib
import socket
import traceback

import numpy as np
import cv2
from PIL import Image, ImageOps

import psycopg2
import psycopg2.extras
import gradio as gr

from dotenv import load_dotenv
load_dotenv()

LABELS = ["NM", "EX", "GD", "LP", "PL", "PO"]

IMG_H = 352
IMG_W = 256

# ✅ Wichtig: Quad etwas größer machen, damit Rand/Ecken nicht abgeschnitten werden
QUAD_EXPAND = 1.04   # typischer guter Start: 1.02 - 1.06

# Rounded mask bleibt optional (Empfehlung: erstmal NICHT anwenden, um Ecken-Schäden sichtbar zu lassen)
CORNER_RADIUS_PX = int(round(IMG_W * 0.05))
APPLY_ROUNDED_MASK_TO_IMAGE = False

# Multi-Views Preview (wird im UI angezeigt; nicht in DB gespeichert)
CROP_FRAC = 0.62


In [3]:
# Schritt 3: DB Verbindung (optional, falls nicht vorhanden)
import os

def get_conn():
    return psycopg2.connect(
        host=os.getenv("PGHOST", "localhost"),
        port=int(os.getenv("PGPORT", "5434")),
        dbname=os.getenv("PGDATABASE", "sam1988"),
        user=os.getenv("PGUSER", "sam1988"),
        password=os.getenv("PGPASSWORD", "Ss190488!")
    )


In [4]:
# Schritt 4: Schema sicherstellen + Migration (damit neue Spalten existieren)
def ensure_schema():
    labels_sql = ",".join([f"'{l}'" for l in LABELS])

    with get_conn() as conn:
        with conn.cursor() as cur:
            # Basis-Tabelle (nur neu, wenn nicht existiert)
            cur.execute(f"""
            CREATE TABLE IF NOT EXISTS pokemon_card_back_samples (
                id UUID PRIMARY KEY,
                created_at TIMESTAMPTZ NOT NULL DEFAULT NOW(),

                label TEXT NOT NULL CHECK (label IN ({labels_sql})),
                note TEXT,

                raw_sha256 TEXT NOT NULL UNIQUE,
                raw_format TEXT NOT NULL,
                raw_w INT,
                raw_h INT,
                raw_bytes BYTEA NOT NULL,

                proc_format TEXT NOT NULL,
                proc_w INT NOT NULL,
                proc_h INT NOT NULL,
                proc_bytes BYTEA NOT NULL
            );
            """)

            # Mask-Spalten (Migration)
            cur.execute("ALTER TABLE pokemon_card_back_samples ADD COLUMN IF NOT EXISTS proc_mask_format TEXT;")
            cur.execute("ALTER TABLE pokemon_card_back_samples ADD COLUMN IF NOT EXISTS proc_mask_w INT;")
            cur.execute("ALTER TABLE pokemon_card_back_samples ADD COLUMN IF NOT EXISTS proc_mask_h INT;")
            cur.execute("ALTER TABLE pokemon_card_back_samples ADD COLUMN IF NOT EXISTS proc_mask_bytes BYTEA;")

            cur.execute("ALTER TABLE pokemon_card_back_samples ALTER COLUMN proc_mask_format SET DEFAULT 'png';")
            cur.execute(f"ALTER TABLE pokemon_card_back_samples ALTER COLUMN proc_mask_w SET DEFAULT {IMG_W};")
            cur.execute(f"ALTER TABLE pokemon_card_back_samples ALTER COLUMN proc_mask_h SET DEFAULT {IMG_H};")

            # ✅ Debug-Spalten (Migration) – für neue Extraktion
            cur.execute("ALTER TABLE pokemon_card_back_samples ADD COLUMN IF NOT EXISTS proc_method TEXT;")
            cur.execute("ALTER TABLE pokemon_card_back_samples ADD COLUMN IF NOT EXISTS proc_quad_expand REAL;")

            # Indizes
            cur.execute("""
                CREATE INDEX IF NOT EXISTS idx_pokemon_card_back_samples_label
                ON pokemon_card_back_samples(label);
            """)
            cur.execute("""
                CREATE INDEX IF NOT EXISTS idx_pokemon_card_back_samples_created_at
                ON pokemon_card_back_samples(created_at DESC);
            """)

def db_counts():
    with get_conn() as conn:
        with conn.cursor(cursor_factory=psycopg2.extras.RealDictCursor) as cur:
            cur.execute("""
                SELECT label, COUNT(*)::int AS n
                FROM pokemon_card_back_samples
                GROUP BY label
                ORDER BY label;
            """)
            rows = cur.fetchall()

    counts = {r["label"]: r["n"] for r in rows}
    for l in LABELS:
        counts.setdefault(l, 0)
    counts["TOTAL"] = sum(counts[l] for l in LABELS)
    return counts

ensure_schema()
print("✅ Schema/Migration OK. Counts:", db_counts())


✅ Schema/Migration OK. Counts: {'EX': 10, 'GD': 10, 'LP': 5, 'NM': 10, 'PL': 5, 'PO': 10, 'TOTAL': 50}


In [5]:
# Schritt 5: Preprocessing-Funktionen (Extraktion + Debug + Multi-Views)
def pil_to_jpeg_bytes(pil_img: Image.Image, quality: int = 92) -> bytes:
    pil_img = ImageOps.exif_transpose(pil_img).convert("RGB")
    buf = io.BytesIO()
    pil_img.save(buf, format="JPEG", quality=quality, optimize=True)
    return buf.getvalue()

def bgr_from_pil(pil_img: Image.Image) -> np.ndarray:
    rgb = np.array(ImageOps.exif_transpose(pil_img).convert("RGB"))
    return cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR)

def pil_from_bgr(bgr: np.ndarray) -> Image.Image:
    rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
    return Image.fromarray(rgb)

def order_points(pts):
    pts = np.array(pts, dtype=np.float32)
    rect = np.zeros((4, 2), dtype=np.float32)
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]      # tl
    rect[2] = pts[np.argmax(s)]      # br
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]   # tr
    rect[3] = pts[np.argmax(diff)]   # bl
    return rect

def expand_quad(quad: np.ndarray, scale: float = 1.04) -> np.ndarray:
    q = quad.astype(np.float32)
    c = q.mean(axis=0, keepdims=True)
    return (c + (q - c) * scale).astype(np.float32)

def warp_quad(bgr, quad, out_w=IMG_W, out_h=IMG_H):
    rect = order_points(quad)
    dst = np.array([[0,0],[out_w-1,0],[out_w-1,out_h-1],[0,out_h-1]], dtype=np.float32)
    M = cv2.getPerspectiveTransform(rect, dst)
    return cv2.warpPerspective(
        bgr, M, (out_w, out_h),
        flags=cv2.INTER_LINEAR,
        borderMode=cv2.BORDER_REFLECT
    )

def try_extract_by_blue_mask(bgr):
    hsv = cv2.cvtColor(bgr, cv2.COLOR_BGR2HSV)
    lower = np.array([80, 40, 40], dtype=np.uint8)
    upper = np.array([150, 255, 255], dtype=np.uint8)
    mask = cv2.inRange(hsv, lower, upper)

    k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11, 11))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, k, iterations=2)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN,  k, iterations=1)

    cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not cnts:
        return None

    cnt = max(cnts, key=cv2.contourArea)
    area = cv2.contourArea(cnt)
    if area < 0.10 * (bgr.shape[0] * bgr.shape[1]):
        return None

    peri = cv2.arcLength(cnt, True)
    approx = cv2.approxPolyDP(cnt, 0.02 * peri, True)
    if len(approx) == 4:
        return approx.reshape(-1, 2)

    rect = cv2.minAreaRect(cnt)
    return cv2.boxPoints(rect)

def try_extract_by_edges(bgr):
    gray = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(gray, 50, 150)
    edges = cv2.dilate(edges, None, iterations=2)

    cnts, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not cnts:
        return None

    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)[:10]
    for cnt in cnts:
        area = cv2.contourArea(cnt)
        if area < 0.10 * (bgr.shape[0] * bgr.shape[1]):
            continue
        peri = cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, 0.02 * peri, True)
        if len(approx) == 4:
            return approx.reshape(-1, 2)
    return None

def overlay_quad_debug(bgr, quad):
    dbg = bgr.copy()
    q = order_points(quad).astype(int)
    cv2.polylines(dbg, [q], isClosed=True, color=(0, 255, 0), thickness=4)
    for (x, y) in q:
        cv2.circle(dbg, (x, y), 10, (0, 0, 255), -1)
    return dbg

def normalize_to_target(bgr, out_w=IMG_W, out_h=IMG_H):
    quad = try_extract_by_blue_mask(bgr)
    method = "blue_mask"
    if quad is None:
        quad = try_extract_by_edges(bgr)
        method = "edges"

    if quad is None:
        resized = cv2.resize(bgr, (out_w, out_h), interpolation=cv2.INTER_AREA)
        return resized, "fallback_resize", False, None

    quad = expand_quad(np.array(quad), scale=QUAD_EXPAND)  # ✅ Randtreuer
    dbg = overlay_quad_debug(bgr, quad)

    warped = warp_quad(bgr, quad, out_w=out_w, out_h=out_h)

    # Portrait sicherstellen
    if warped.shape[1] > warped.shape[0]:
        warped = cv2.rotate(warped, cv2.ROTATE_90_CLOCKWISE)

    warped = cv2.resize(warped, (out_w, out_h), interpolation=cv2.INTER_AREA)
    return warped, f"{method}_expand{QUAD_EXPAND}", True, dbg

def encode_png_bytes_from_bgr(bgr):
    ok, enc = cv2.imencode(".png", bgr)
    if not ok:
        raise RuntimeError("PNG encoding failed")
    return enc.tobytes()

def encode_png_bytes_gray(gray: np.ndarray) -> bytes:
    ok, enc = cv2.imencode(".png", gray)
    if not ok:
        raise RuntimeError("PNG encoding failed (mask)")
    return enc.tobytes()

def rounded_rect_mask(h: int, w: int, r: int) -> np.ndarray:
    r = int(max(0, r))
    r = min(r, min(h, w) // 2)
    mask = np.zeros((h, w), dtype=np.uint8)
    if r == 0:
        mask[:] = 255
        return mask

    cv2.rectangle(mask, (r, 0), (w - r - 1, h - 1), 255, -1)
    cv2.rectangle(mask, (0, r), (w - 1, h - r - 1), 255, -1)

    cv2.circle(mask, (r, r), r, 255, -1)
    cv2.circle(mask, (w - r - 1, r), r, 255, -1)
    cv2.circle(mask, (r, h - r - 1), r, 255, -1)
    cv2.circle(mask, (w - r - 1, h - r - 1), r, 255, -1)
    return mask

def make_multiviews(img_rgb_uint8: np.ndarray):
    """
    Views aus dem normalisierten Proc-Bild (IMG_H, IMG_W, 3).
    Enthält:
      - 9 Basis-Views (Full + 4 Ecken + 4 Kanten)
      - +3 Augs für Full (H-Flip, V-Flip, 180°)
      - +12 Augs für Ecken (je Ecke H-Flip, V-Flip, 180°)
    => total 24 Views
    """
    img = img_rgb_uint8
    H, W = img.shape[:2]

    ch = int(round(H * CROP_FRAC))
    cw = int(round(W * CROP_FRAC))

    def crop(y0, x0, y1, x1):
        c = img[y0:y1, x0:x1]
        return cv2.resize(c, (IMG_W, IMG_H), interpolation=cv2.INTER_AREA)

    # Basis
    full = img
    tl = crop(0, 0, ch, cw)
    tr = crop(0, W-cw, ch, W)
    bl = crop(H-ch, 0, H, cw)
    br = crop(H-ch, W-cw, H, W)

    y_mid0 = (H - ch)//2
    x_mid0 = (W - cw)//2
    top = crop(0, x_mid0, ch, x_mid0+cw)
    bottom = crop(H-ch, x_mid0, H, x_mid0+cw)
    left = crop(y_mid0, 0, y_mid0+ch, cw)
    right = crop(y_mid0, W-cw, y_mid0+ch, W)

    base_views = [
        ("full", full),
        ("corner_tl", tl), ("corner_tr", tr), ("corner_bl", bl), ("corner_br", br),
        ("edge_top", top), ("edge_bottom", bottom), ("edge_left", left), ("edge_right", right),
    ]

    def aug(name, v):
        return [
            (name, v),
            (name + "_hflip", cv2.flip(v, 1)),
            (name + "_vflip", cv2.flip(v, 0)),
            (name + "_rot180", cv2.rotate(v, cv2.ROTATE_180)),
        ]

    views_named = []
    # alle Basis-Views ohne Aug (9)
    views_named.extend(base_views)

    # Full augmented (+3)
    views_named.extend(aug("full", full)[1:])

    # Ecken augmented (+12)
    for nm, v in [("corner_tl", tl), ("corner_tr", tr), ("corner_bl", bl), ("corner_br", br)]:
        views_named.extend(aug(nm, v)[1:])

    return views_named  # list[(name, rgb_uint8)]
    
def prepare_for_db(pil_img: Image.Image):
    raw_jpeg = pil_to_jpeg_bytes(pil_img, quality=92)
    raw_pil = Image.open(io.BytesIO(raw_jpeg)).convert("RGB")
    raw_w, raw_h = raw_pil.size

    bgr = bgr_from_pil(raw_pil)
    proc_bgr, method, extracted, dbg_bgr = normalize_to_target(bgr, out_w=IMG_W, out_h=IMG_H)

    mask = rounded_rect_mask(IMG_H, IMG_W, CORNER_RADIUS_PX)
    if APPLY_ROUNDED_MASK_TO_IMAGE:
        proc_bgr = cv2.bitwise_and(proc_bgr, proc_bgr, mask=mask)

    proc_png = encode_png_bytes_from_bgr(proc_bgr)
    mask_png = encode_png_bytes_gray(mask)

    preview_pil = pil_from_bgr(proc_bgr)
    mask_preview_pil = Image.fromarray(mask)
    dbg_preview_pil = pil_from_bgr(dbg_bgr) if dbg_bgr is not None else None
    raw_preview_pil = raw_pil

    # Multi-views (Preview)
    proc_rgb = cv2.cvtColor(proc_bgr, cv2.COLOR_BGR2RGB)
    views_named = make_multiviews(proc_rgb)  # list[(name, rgb)]
    views_gallery = [(Image.fromarray(v), name) for name, v in views_named]

    info = {
        "raw_size": [raw_w, raw_h],
        "proc_size": [IMG_W, IMG_H],
        "method": method,
        "extracted": extracted,
        "quad_expand": QUAD_EXPAND,
        "mask_applied": APPLY_ROUNDED_MASK_TO_IMAGE,
        "views_count": len(views_named),
        "raw_sha256": hashlib.sha256(raw_jpeg).hexdigest()
    }

    return (
        raw_jpeg, "jpeg", raw_w, raw_h,
        proc_png, "png", IMG_W, IMG_H,
        mask_png, "png", IMG_W, IMG_H,
        preview_pil, mask_preview_pil, raw_preview_pil, dbg_preview_pil,
        views_gallery,
        info
    )


In [6]:
# Schritt 6: DB Insert/Fetch + saubere Fehlermeldung
def fmt_pg_error(e: Exception) -> str:
    if isinstance(e, psycopg2.Error):
        parts = [f"{type(e).__name__}: {e}"]
        if getattr(e, "pgcode", None):
            parts.append(f"pgcode: {e.pgcode}")
        if getattr(e, "pgerror", None):
            parts.append(f"pgerror: {e.pgerror}")
        diag = getattr(e, "diag", None)
        if diag is not None:
            for k in ["message_detail", "message_hint", "schema_name", "table_name", "column_name", "constraint_name"]:
                v = getattr(diag, k, None)
                if v:
                    parts.append(f"{k}: {v}")
        return "\n".join(parts)
    return f"{type(e).__name__}: {e}"

def insert_sample(label, note,
                  raw_bytes, raw_format, raw_w, raw_h,
                  proc_bytes, proc_format, proc_w, proc_h,
                  mask_bytes, mask_format, mask_w, mask_h,
                  proc_method, proc_quad_expand):
    raw_sha = hashlib.sha256(raw_bytes).hexdigest()
    sample_id = uuid.uuid4()

    try:
        with get_conn() as conn:
            with conn.cursor() as cur:
                cur.execute("""
                    INSERT INTO pokemon_card_back_samples (
                        id, label, note,
                        raw_sha256, raw_format, raw_w, raw_h, raw_bytes,
                        proc_format, proc_w, proc_h, proc_bytes,
                        proc_mask_format, proc_mask_w, proc_mask_h, proc_mask_bytes,
                        proc_method, proc_quad_expand
                    )
                    VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
                """, (
                    str(sample_id), label, note,
                    raw_sha, raw_format, raw_w, raw_h, psycopg2.Binary(raw_bytes),
                    proc_format, proc_w, proc_h, psycopg2.Binary(proc_bytes),
                    mask_format, mask_w, mask_h, psycopg2.Binary(mask_bytes),
                    proc_method, float(proc_quad_expand) if proc_quad_expand is not None else None
                ))
        return True, raw_sha, "✅ Gespeichert."
    except Exception as e:
        return False, raw_sha, "❌ DB-Fehler:\n" + fmt_pg_error(e) + "\n\n" + traceback.format_exc()

def fetch_recent(limit=24, label=None):
    q = """
        SELECT id, created_at, label, note, proc_bytes, proc_method
        FROM pokemon_card_back_samples
    """
    params = []
    if label and label != "ALL":
        q += " WHERE label = %s"
        params.append(label)
    q += " ORDER BY created_at DESC LIMIT %s"
    params.append(int(limit))

    with get_conn() as conn:
        with conn.cursor(cursor_factory=psycopg2.extras.RealDictCursor) as cur:
            cur.execute(q, params)
            return cur.fetchall()


In [7]:
# Schritt 7: Gradio UI für Handy-Upload & DB-Speicherung
def save_from_ui(pil_img, label, note):
    if pil_img is None:
        return "❌ Bitte ein Bild hochladen.", None, None, None, None, [], db_counts()

    if label not in LABELS:
        return f"❌ Ungültiges Label: {label}", None, None, None, None, [], db_counts()

    (raw_bytes, raw_fmt, raw_w, raw_h,
     proc_bytes, proc_fmt, proc_w, proc_h,
     mask_bytes, mask_fmt, mask_w, mask_h,
     preview_pil, mask_preview_pil, raw_preview_pil, dbg_preview_pil,
     views_gallery,
     info) = prepare_for_db(pil_img)

    ok, sha, msg = insert_sample(
        label=label,
        note=(note or "").strip() or None,
        raw_bytes=raw_bytes, raw_format=raw_fmt, raw_w=raw_w, raw_h=raw_h,
        proc_bytes=proc_bytes, proc_format=proc_fmt, proc_w=proc_w, proc_h=proc_h,
        mask_bytes=mask_bytes, mask_format=mask_fmt, mask_w=mask_w, mask_h=mask_h,
        proc_method=info.get("method"),
        proc_quad_expand=info.get("quad_expand"),
    )

    status = {
        "status": "saved" if ok else "failed",
        "message": msg,
        "label": label,
        "raw_sha256": sha,
        "prep": info
    }
    return (
        json.dumps(status, ensure_ascii=False, indent=2),
        raw_preview_pil,
        dbg_preview_pil,
        preview_pil,
        mask_preview_pil,
        views_gallery,
        db_counts()
    )

def load_gallery(label, limit):
    rows = fetch_recent(limit=limit, label=label)
    items = []
    for r in rows:
        proc_bytes = bytes(r["proc_bytes"])
        pil = Image.open(io.BytesIO(proc_bytes)).convert("RGB")
        pm = r.get("proc_method") or "old"
        cap = f'{r["label"]} | {pm} | {r["created_at"].strftime("%Y-%m-%d %H:%M")} | {str(r["id"])[:8]}'
        items.append((pil, cap))
    return items

with gr.Blocks(title="Pokemon Card Back Uploader") as app:
    gr.Markdown("# 📸 Notebook 1: Upload & Label (randtreuer Warp + Multi-View Preview inkl. Spiegel/Umkehr)")

    with gr.Tab("Upload"):
        with gr.Row():
            img_in = gr.Image(label="Foto hochladen (Rückseite)", type="pil")

        with gr.Row():
            raw_prev = gr.Image(label="Raw (Original)", type="pil")
            dbg_prev = gr.Image(label="Debug: Quad (grün) + Punkte (rot)", type="pil")

        with gr.Row():
            proc_prev = gr.Image(label="Proc (Warped/Normalized)", type="pil")
            mask_prev = gr.Image(label="Mask (gespeichert; optional angewandt)", type="pil")

        with gr.Row():
            dd = gr.Dropdown(choices=LABELS, value="NM", label="Zustandsklasse")
            note = gr.Textbox(label="Notiz (optional)", placeholder="z.B. 'gutes Licht', 'schief', ...")

        btn = gr.Button("In DB speichern", variant="primary")
        out = gr.Code(label="Status (JSON)", language="json")
        stats = gr.JSON(label="DB Counts")

        gr.Markdown("## 👀 Multi-View Preview (24 Views: Crops + Spiegeln/Umkehr/Drehen)")
        views_gal = gr.Gallery(label="Views", columns=6, height="auto")

        btn.click(
            fn=save_from_ui,
            inputs=[img_in, dd, note],
            outputs=[out, raw_prev, dbg_prev, proc_prev, mask_prev, views_gal, stats]
        )
        stats.value = db_counts()

    with gr.Tab("Browse"):
        with gr.Row():
            gal_label = gr.Dropdown(choices=["ALL"] + LABELS, value="ALL", label="Filter")
            gal_limit = gr.Slider(6, 60, value=24, step=1, label="Anzahl")
            gal_btn = gr.Button("Aktualisieren")

        gallery = gr.Gallery(label="Letzte Uploads", columns=4, height="auto")
        gal_btn.click(fn=load_gallery, inputs=[gal_label, gal_limit], outputs=[gallery])
        gallery.value = load_gallery("ALL", 24)


In [8]:
# Schritt 8: Server starten (Handy im gleichen WLAN)
def guess_local_ip():
    s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    try:
        s.connect(("10.255.255.255", 1))
        ip = s.getsockname()[0]
    except Exception:
        ip = "127.0.0.1"
    finally:
        s.close()
    return ip

def get_free_port():
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind(("", 0))
    port = s.getsockname()[1]
    s.close()
    return port

local_ip = guess_local_ip()
port = get_free_port()

print(f"👉 Öffne am Handy (gleiches WLAN): http://{local_ip}:{port}")
app.launch(server_name="0.0.0.0", server_port=port, share=False)


👉 Öffne am Handy (gleiches WLAN): http://192.168.8.10:50013
* Running on local URL:  http://0.0.0.0:50013
* To create a public link, set `share=True` in `launch()`.
